In [1]:
%pylab --no-import-all inline

import EKPSolvers
import gurobipy as grb



Populating the interactive namespace from numpy and matplotlib


In [3]:
a = EKPSolvers.summing(4,5)
print('Ein Test',a)

Ein Test 9


In [6]:
x = [1,2,3,4]
y = [1,4,7,10]

model = grb.Model("GradientSparse")
